In [2]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-65m_w_45/unsloth_634d431104d14e81bec24370de4e8d63
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-65m_w_45/unsloth_634d431104d14e81bec24370de4e8d63
  Resolved https://github.com/unslothai/unsloth.git to commit 52bc19d1fa4cd3557b785127fd68b5f4d1c34347
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 9.5 MB/s eta 0:00:00
   ━━

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype=None
load_in_4bit = True
##loading the model from the unsloth library
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

In [5]:
##now we will run LoRA for this model, r = 16 means the rank, higher the rank - higher the impact will be (more changeabe parameters in the Low Rank Matrices)
##the lora_alpha will determine the impact or how much magintude of the change will be upon the new formed matrix
model = FastLanguageModel.get_peft_model(
model,
r=16,
target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
lora_alpha=16,
lora_dropout=0,
bias="none",
use_gradient_checkpointing="unsloth",
 random_state=3407,
 use_rslora=False,
 loftq_config=None)


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
##Above is the Alpaca format
##the model will keep generating text until it runs out of tokens, this EOS token makes sure it gives a specific output and then stops
EOS_TOKEN=tokenizer.eos_token
def formatting_prompts_func(examples):
  instructions = examples['instruction']
  input= examples['input']
  output=examples['output']
  texts=[]
  for instruction, input, output in zip(instructions, input, output):
    text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
    texts.append(text)
  return {"texts":texts,}
pass


from datasets import load_dataset
##today we will be working with the finance-alpaca dataset where the llama3.1 8b model will be finetuned on a finance dataset
ds = load_dataset("gbharti/finance-alpaca", split="train")
dataset = ds.map(formatting_prompts_func, batched=True,)


Generating train split:   0%|          | 0/68912 [00:00<?, ? examples/s]

Map:   0%|          | 0/68912 [00:00<?, ? examples/s]

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

##running the Supervised finetuning trainer with its hyperparameters
trainer = SFTTrainer(
    model = model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="texts",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args = TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps=1,
        optim = "adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir="outputs"
    ),
)


Map (num_proc=2):   0%|          | 0/68912 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 68,912 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.876900
2,2.722900
3,2.687800
4,2.544200
5,1.980000
6,1.991400
7,1.876800
8,1.599500
9,1.327800
10,1.159300


In [11]:
##enabling inference using the for_inference function in unsloth
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    ##this is in the alpaca format too
    alpaca_prompt.format(
        "What is beta in finance, describe it in a creative format?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is beta in finance, describe it in a creative format?

### Input:


### Response:
Beta is the measure of a stock's volatility relative to the overall market. It's like a compass that helps investors navigate the financial landscape. It's the equivalent of a compass in finance, helping investors navigate the financial landscape.<|end_of_text|>


In [10]:
#saving the pretrained model
model.save_pretrained("llama3.1b-finance-fintuned")
tokenizer.save_pretrained("llama3.1b-finance-fintuned")

('llama3.1b-finance-fintuned/tokenizer_config.json',
 'llama3.1b-finance-fintuned/special_tokens_map.json',
 'llama3.1b-finance-fintuned/tokenizer.json')